# Evolutionary Algorithms - Exercise 4.2

Setup for permutation-based optimization using Function6


## Setup and Imports


In [ ]:
import requests
import numpy as np
from typing import List, Tuple, Callable
import matplotlib.pyplot as plt


In [ ]:
class BlackBox:
    """
    Interface to the black-box service for ODM course.
    """
    def __init__(self, token: int, endpoint: str = 'http://ls-stat-ml.uni-muenster.de:7300/'):
        self.endpoint = endpoint
        self.token = token

    def evaluate(self, objective: str, parameters: list) -> float:
        r = requests.post(url=self.endpoint + "compute/" + objective,
                          json={"parameters": [str(v) for v in parameters], "token": self.token})
        if r.status_code == 200:
            return float(r.json())
        else:
            raise ValueError(r)


In [ ]:
group_number = 11
bb = BlackBox(token=group_number)

REFERENCE_SOLUTION = list(range(0, 20))
reference_fitness = bb.evaluate("Function6", REFERENCE_SOLUTION)
print(f"Reference solution [0,1,2,...,19] has fitness: {reference_fitness}")


## Core EA Components


### Population Management


In [ ]:
def initialize_population(population_size: int, permutation_length: int = 20) -> List[np.ndarray]:
    """
    Create initial population of random permutations.
    
    Args:
        population_size: Number of individuals in population
        permutation_length: Length of each permutation (default 20 for Function6)
    
    Returns:
        List of numpy arrays, each representing a permutation
    """
    return [np.random.permutation(permutation_length) for _ in range(population_size)]


In [ ]:
population = initialize_population(population_size=10)
print(f"Created population with {len(population)} individuals")
print(f"Example individual: {population[0]}")


### Central Evaluation Function


In [ ]:
def evaluate_individual(individual: np.ndarray, blackbox: BlackBox, function_name: str = "Function6") -> float:
    """
    Central evaluation function for a single individual.
    
    Args:
        individual: Permutation to evaluate
        blackbox: BlackBox instance for evaluation
        function_name: Name of the function to evaluate (default: Function6)
    
    Returns:
        Fitness value (lower is better for minimization)
    """
    return blackbox.evaluate(function_name, individual.tolist())


def evaluate_population(population: List[np.ndarray], blackbox: BlackBox, function_name: str = "Function6") -> List[float]:
    """
    Evaluate entire population.
    
    Args:
        population: List of individuals
        blackbox: BlackBox instance for evaluation
        function_name: Name of the function to evaluate
    
    Returns:
        List of fitness values corresponding to each individual
    """
    return [evaluate_individual(ind, blackbox, function_name) for ind in population]


In [ ]:
fitness_values = evaluate_population(population, bb)
print(f"Fitness values: {fitness_values}")
print(f"Best fitness in population: {min(fitness_values)}")
print(f"Worst fitness in population: {max(fitness_values)}")


### Selection Operators


In [ ]:
def tournament_selection(population: List[np.ndarray], fitness_values: List[float], tournament_size: int = 3) -> np.ndarray:
    """
    Select individual using tournament selection.
    
    Args:
        population: List of individuals
        fitness_values: Corresponding fitness values
        tournament_size: Number of individuals in tournament
    
    Returns:
        Selected individual
    """
    indices = np.random.choice(len(population), size=tournament_size, replace=False)
    tournament_fitness = [fitness_values[i] for i in indices]
    winner_idx = indices[np.argmin(tournament_fitness)]
    return population[winner_idx].copy()


def roulette_wheel_selection(population: List[np.ndarray], fitness_values: List[float]) -> np.ndarray:
    """
    Select individual using fitness-proportional selection (for minimization).
    
    Args:
        population: List of individuals
        fitness_values: Corresponding fitness values
    
    Returns:
        Selected individual
    """
    inverted_fitness = [1.0 / (1.0 + f) for f in fitness_values]
    probabilities = np.array(inverted_fitness) / sum(inverted_fitness)
    selected_idx = np.random.choice(len(population), p=probabilities)
    return population[selected_idx].copy()


### Mutation Operators


In [ ]:
def swap_mutation(individual: np.ndarray) -> np.ndarray:
    """
    Swap two random positions in permutation.
    
    Args:
        individual: Permutation to mutate
    
    Returns:
        Mutated permutation
    """
    mutated = individual.copy()
    i, j = np.random.choice(len(mutated), size=2, replace=False)
    mutated[i], mutated[j] = mutated[j], mutated[i]
    return mutated


def insert_mutation(individual: np.ndarray) -> np.ndarray:
    """
    Remove element at one position and insert at another.
    
    Args:
        individual: Permutation to mutate
    
    Returns:
        Mutated permutation
    """
    mutated = individual.copy()
    i, j = np.random.choice(len(mutated), size=2, replace=False)
    element = mutated[i]
    mutated = np.delete(mutated, i)
    mutated = np.insert(mutated, j, element)
    return mutated


def inversion_mutation(individual: np.ndarray) -> np.ndarray:
    """
    Reverse a random segment of the permutation.
    
    Args:
        individual: Permutation to mutate
    
    Returns:
        Mutated permutation
    """
    mutated = individual.copy()
    i, j = sorted(np.random.choice(len(mutated), size=2, replace=False))
    mutated[i:j+1] = mutated[i:j+1][::-1]
    return mutated


### Crossover Operators


In [ ]:
def order_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Order crossover (OX) for permutations.
    
    Args:
        parent1: First parent permutation
        parent2: Second parent permutation
    
    Returns:
        Tuple of two offspring
    """
    size = len(parent1)
    i, j = sorted(np.random.choice(size, size=2, replace=False))
    
    offspring1 = np.full(size, -1)
    offspring1[i:j+1] = parent1[i:j+1]
    
    offspring2 = np.full(size, -1)
    offspring2[i:j+1] = parent2[i:j+1]
    
    def fill_offspring(offspring, donor):
        pos = (j + 1) % size
        for gene in np.roll(donor, -j-1):
            if gene not in offspring:
                offspring[pos] = gene
                pos = (pos + 1) % size
        return offspring
    
    offspring1 = fill_offspring(offspring1, parent2)
    offspring2 = fill_offspring(offspring2, parent1)
    
    return offspring1, offspring2


def pmx_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Partially Mapped Crossover (PMX) for permutations.
    
    Args:
        parent1: First parent permutation
        parent2: Second parent permutation
    
    Returns:
        Tuple of two offspring
    """
    size = len(parent1)
    i, j = sorted(np.random.choice(size, size=2, replace=False))
    
    offspring1 = parent1.copy()
    offspring2 = parent2.copy()
    
    for k in range(i, j+1):
        val1, val2 = offspring1[k], offspring2[k]
        offspring1[k], offspring2[k] = val2, val1
        
        idx1 = np.where(offspring1 == val2)[0]
        idx2 = np.where(offspring2 == val1)[0]
        
        if len(idx1) > 1:
            other_idx = idx1[idx1 != k][0]
            offspring1[other_idx] = val1
            
        if len(idx2) > 1:
            other_idx = idx2[idx2 != k][0]
            offspring2[other_idx] = val2
    
    return offspring1, offspring2


### Circular Crossover (for Exercise 4.2)


In [ ]:
def circular_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Circular crossover - placeholder for implementation.
    TODO: Implement according to course material.
    
    Args:
        parent1: First parent permutation
        parent2: Second parent permutation
    
    Returns:
        Tuple of two offspring
    """
    return order_crossover(parent1, parent2)


## Evolutionary Algorithm Framework


In [ ]:
def evolutionary_algorithm(
    blackbox: BlackBox,
    population_size: int,
    generations: int,
    mutation_rate: float = 0.1,
    crossover_rate: float = 0.8,
    tournament_size: int = 3,
    mutation_operator: Callable = swap_mutation,
    crossover_operator: Callable = order_crossover,
    selection_operator: Callable = tournament_selection,
    permutation_length: int = 20,
    verbose: bool = True
) -> Tuple[np.ndarray, float, List[float]]:
    """
    Generic evolutionary algorithm framework for permutation problems.
    
    Args:
        blackbox: BlackBox instance for evaluation
        population_size: Size of population
        generations: Number of generations to evolve
        mutation_rate: Probability of mutation
        crossover_rate: Probability of crossover
        tournament_size: Size for tournament selection
        mutation_operator: Function for mutation
        crossover_operator: Function for crossover
        selection_operator: Function for selection
        permutation_length: Length of permutations
        verbose: Print progress information
    
    Returns:
        Tuple of (best_individual, best_fitness, fitness_history)
    """
    population = initialize_population(population_size, permutation_length)
    fitness_values = evaluate_population(population, blackbox)
    
    best_idx = np.argmin(fitness_values)
    best_individual = population[best_idx].copy()
    best_fitness = fitness_values[best_idx]
    
    fitness_history = [best_fitness]
    
    for generation in range(generations):
        new_population = []
        
        while len(new_population) < population_size:
            if np.random.random() < crossover_rate:
                parent1 = selection_operator(population, fitness_values, tournament_size) if selection_operator == tournament_selection else selection_operator(population, fitness_values)
                parent2 = selection_operator(population, fitness_values, tournament_size) if selection_operator == tournament_selection else selection_operator(population, fitness_values)
                
                offspring1, offspring2 = crossover_operator(parent1, parent2)
                new_population.extend([offspring1, offspring2])
            else:
                parent = selection_operator(population, fitness_values, tournament_size) if selection_operator == tournament_selection else selection_operator(population, fitness_values)
                new_population.append(parent)
        
        new_population = new_population[:population_size]
        
        for i in range(len(new_population)):
            if np.random.random() < mutation_rate:
                new_population[i] = mutation_operator(new_population[i])
        
        population = new_population
        fitness_values = evaluate_population(population, blackbox)
        
        current_best_idx = np.argmin(fitness_values)
        current_best_fitness = fitness_values[current_best_idx]
        
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_individual = population[current_best_idx].copy()
        
        fitness_history.append(best_fitness)
        
        if verbose and (generation + 1) % 10 == 0:
            avg_fitness = np.mean(fitness_values)
            print(f"Generation {generation + 1}/{generations}: Best={best_fitness:.4f}, Avg={avg_fitness:.4f}")
    
    return best_individual, best_fitness, fitness_history


## Example Usage


In [ ]:
best_solution, best_fitness, history = evolutionary_algorithm(
    blackbox=bb,
    population_size=20,
    generations=50,
    mutation_rate=0.2,
    crossover_rate=0.8,
    tournament_size=3,
    mutation_operator=swap_mutation,
    crossover_operator=order_crossover,
    selection_operator=tournament_selection,
    verbose=True
)

print(f"\n{'='*50}")
print(f"Best solution found: {best_solution}")
print(f"Best fitness: {best_fitness:.4f}")
print(f"Reference fitness: {reference_fitness:.4f}")
print(f"Improvement: {reference_fitness - best_fitness:.4f}")


### Visualization


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history, linewidth=2)
plt.axhline(y=reference_fitness, color='r', linestyle='--', label='Reference Solution')
plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Evolutionary Algorithm Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


## Experiment Template

Use this section to experiment with different configurations:
.8

In [ ]:
experiments = {
    "Small Population": {"population_size": 10, "generations": 100},
    "Large Population": {"population_size": 50, "generations": 50},
    "High Mutation": {"population_size": 20, "generations": 50, "mutation_rate": 0.5},
    "Low Mutation": {"population_size": 20, "generations": 50, "mutation_rate": 0.05},
}

results = {}

for name, config in experiments.items():
    print(f"\n{'='*50}")
    print(f"Running: {name}")
    print(f"{'='*50}")
    
    best_sol, best_fit, hist = evolutionary_algorithm(
        blackbox=bb,
        **config,
        verbose=False
    )
    
    results[name] = {
        "solution": best_sol,
        "fitness": best_fit,
        "history": hist
    }
    
    print(f"Best fitness: {best_fit:.4f}")
    print(f"Improvement over reference: {reference_fitness - best_fit:.4f}")


In [ ]:
plt.figure(figsize=(12, 6))
for name, data in results.items():
    plt.plot(data["history"], label=name, linewidth=2)

plt.axhline(y=reference_fitness, color='r', linestyle='--', label='Reference Solution', linewidth=2)
plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Comparison of Different EA Configurations')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


## Your Experiments

Use the space below to test different operators and configurations:


In [ ]:
# Your code here
